In [34]:
import pandas as pd
import tensorflow as tf
import keras as ks
import numpy as np
from keras.layers import Dense, Dropout, Conv2D, Flatten, Activation
from keras.models import Sequential
import matplotlib.pylab as plt

In [3]:
# Create new variables to be used in Keras and the CNN

# number of items to use for training
BATCH_SIZE = 10 

# Number of identifying classes 
#   WE have two, Bloom and no bloom 1/0
NUM_CLASSES = 2 

# number of times to repeat process
EPOCHS = 20

df = pd.read_csv('../../data/cleaned/site1_malin.csv')
df.head()

,date (mm.dd.yyyy),Depth [m],T [deg C],DO [%],SPC [micro S/cm],NH4 [mg/l],NO3 [mg/l],PO4 [mg/l],Chl tot [micro g/l],Chl cyano [micro g/l],Chl green Al. [micro g/l],turb [FTU],Secci Depth [m],pH
0,6/1/2016,0.0,20.8,127.3,748,0.877326755,0.162558848,0.012251796,39.8,3.2,36.6,5.1,0.8,8.63
1,6/1/2016,0.5,20.8,127.0,748,-,-,-,55.4,3.7,51.7,4.8,-,8.65
2,6/1/2016,1.0,20.4,131.4,756,-,-,-,45.6,3.0,42.6,4.9,-,8.66
3,6/1/2016,2.0,18.0,85.4,775,-,-,-,41.0,3.9,37.1,5.0,-,8.25
4,6/1/2016,3.0,15.8,4.0,786,1.050011213,0.270562171,0.031263202,54.6,5.8,48.8,5.8,-,7.89


In [48]:
target = df['Chl cyano [micro g/l]'].apply(lambda x : 1 if x > 20 else 0)
dataset = df.drop(columns=['NH4 [mg/l]', 'NO3 [mg/l]', 'PO4 [mg/l]', 'Secci Depth [m]','Chl cyano [micro g/l]'])
dataset['bloom'] = target
dataset

,date (mm.dd.yyyy),Depth [m],T [deg C],DO [%],SPC [micro S/cm],Chl tot [micro g/l],Chl green Al. [micro g/l],turb [FTU],pH,bloom
0,6/1/2016,0.0,20.8,127.3,748,39.8,36.6,5.1,8.63,0
1,6/1/2016,0.5,20.8,127.0,748,55.4,51.7,4.8,8.65,0
2,6/1/2016,1.0,20.4,131.4,756,45.6,42.6,4.9,8.66,0
3,6/1/2016,2.0,18.0,85.4,775,41.0,37.1,5.0,8.25,0
4,6/1/2016,3.0,15.8,4.0,786,54.6,48.8,5.8,7.89,0
5,6/8/2016,0.0,22.5,76.6,762,5.3,4.4,5.3,8.17,0
6,6/8/2016,0.5,22.5,77.0,763,6.9,6.0,4.3,8.18,0
7,6/8/2016,1.0,22.5,73.2,762,8.5,7.6,4.1,8.06,0
8,6/8/2016,2.0,21.0,33.1,776,19.6,17.2,4.7,7.90,0
9,6/8/2016,3.0,17.2,2.4,738,78.3,65.8,16.5,7.51,0


In [5]:
from datetime import datetime

dataset['date (mm.dd.yyyy)'] = pd.to_datetime(dataset['date (mm.dd.yyyy)'])
dataset['date (mm.dd.yyyy)'] = (dataset['date (mm.dd.yyyy)'] - dataset['date (mm.dd.yyyy)'].min()) / np.timedelta64(1,'D')
dataset.dtypes

date (mm.dd.yyyy)            float64
Depth [m]                    float64
T [deg C]                    float64
DO [%]                       float64
SPC [micro S/cm]               int64
Chl tot [micro g/l]          float64
Chl green Al. [micro g/l]    float64
turb [FTU]                   float64
pH                           float64
bloom                          int64
dtype: object

In [6]:
# lets try to normalize this now....
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
ds_scaled = scaler.fit_transform(dataset)
dataset = pd.DataFrame(ds_scaled,columns=['date (mm.dd.yyyy)','Depth [m]','T [deg C]', 'DO [%]','SPC [micro S/cm] ',
'Chl tot [micro g/l]','Chl green Al. [micro g/l]',
'turb [FTU]','pH ','bloom'])
dataset.describe()

/home/mytstikyl/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,date (mm.dd.yyyy),Depth [m],T [deg C],DO [%],SPC [micro S/cm],Chl tot [micro g/l],Chl green Al. [micro g/l],turb [FTU],pH,bloom
count,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
mean,0.500000,0.433333,0.630545,0.577551,0.402597,0.332460,0.260335,0.137371,0.483394,0.300000
std,0.303492,0.360654,0.249957,0.205032,0.206691,0.190768,0.212422,0.176529,0.211168,0.460355
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.238095,0.166667,0.556667,0.526077,0.219780,0.219836,0.107899,0.034502,0.316667,0.000000
50%,0.500000,0.333333,0.713333,0.590703,0.357143,0.279141,0.192182,0.059955,0.460000,0.000000
75%,0.761905,0.666667,0.771667,0.670068,0.571429,0.374233,0.333062,0.160633,0.680000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
# we need to transform this data into an synthetic image


ValueError: Negative dimension size caused by subtracting 5 from 1 for 'conv2d_4/convolution' (op: 'Conv2D') with input shapes: [?,1,77,2], [5,2,2,32].

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

ValueError: cannot reshape array of size 693 into shape (1,1,77,1)

ValueError: Input 0 is incompatible with layer conv2d_1: expected ndim=4, found ndim=3